In [ ]:
!pip install -U langchain-community pypdf langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_api')

In [ ]:
# Get a PDF
!wget https://arxiv.org/pdf/2402.11753.pdf -q
# Load PDF
loader = PyPDFLoader(file_path = "2402.11753.pdf")
pages = loader.load_and_split()

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Stuff

In [ ]:
def stuff_summary(pages):
  ## Define prompt
  prompt_template = """
  Write a concise summary of the paper below, avoid using technical words, list the main points of the paper in bullet points, within 200 words.
  ```{paper}```
  """
  prompt = PromptTemplate.from_template(prompt_template)

  # Define LLM chain
  llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", api_key=open_ai_key)
  llm_chain = LLMChain(llm=llm, prompt=prompt)



  # Define StuffDocumentsChain
  stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="paper")

  stuff_result = stuff_chain.invoke(pages)

  return stuff_result['output_text']

In [ ]:
stuff_summary(pages)

"**Summary of the Paper:**\n\nThis paper discusses vulnerabilities in large language models (LLMs) related to safety measures. It introduces a new attack method called ArtPrompt, which uses ASCII art to bypass these safety protocols. The authors highlight that existing safety techniques focus only on the meaning of words, ignoring how they can be represented visually. This oversight allows malicious users to exploit LLMs by using ASCII art to mask harmful instructions. The paper presents a benchmark called the Vision-in-Text Challenge (VITC) to evaluate LLMs' ability to recognize ASCII art prompts. The results show that popular LLMs struggle with this task, making them susceptible to the ArtPrompt attack. The authors demonstrate that ArtPrompt can effectively induce unsafe behaviors in multiple LLMs, even when they are equipped with safety measures.\n\n**Main Points:**\n- Safety in LLMs is crucial but currently inadequate.\n- Existing safety measures focus only on word meanings.\n- ASC

**Summary of the Paper:**

This paper discusses vulnerabilities in large language models (LLMs) related to safety measures. It introduces a new attack method called ArtPrompt, which uses ASCII art to bypass these safety protocols. The authors highlight that existing safety techniques focus only on the meaning of words, ignoring how they can be represented visually. This oversight allows malicious users to exploit LLMs by using ASCII art to mask harmful instructions. The paper presents a benchmark called the Vision-in-Text Challenge (VITC) to evaluate LLMs' ability to recognize ASCII art prompts. The results show that popular LLMs struggle with this task, making them susceptible to the ArtPrompt attack. The authors demonstrate that ArtPrompt can effectively induce unsafe behaviors in multiple LLMs, even when they are equipped with safety measures.

**Main Points:**
- Safety in LLMs is crucial but currently inadequate.
- Existing safety measures focus only on word meanings.
- ASCII art can be used to bypass these safety measures.
- The Vision-in-Text Challenge (VITC) benchmark evaluates LLMs' recognition of ASCII art.
- Five state-of-the-art LLMs struggle with ASCII art prompts.
- ArtPrompt effectively induces unsafe behaviors in LLMs.
- The attack can be executed with minimal effort and without direct access to the models.

#Map Reduce

In [ ]:
# Start LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", api_key=open_ai_key)

# Prompt for each chunk
map_prompt_template = """
Write a concise summary of the paper below, list the main points of the paper in bullet points, within 200 words.
```{text}```
"""
map_prompt = PromptTemplate.from_template(map_prompt_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Prompt for final sumamrization
reduce_prompt_template = """
You are given multiple summaries from different sections of a document.
Avoid using technical words, combine them and list the main points of the paper in bullet points, within 200 words.
```{text}```
"""
reduce_prompt = PromptTemplate.from_template(reduce_prompt_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Use StuffDocumentsChain to combine summaries for small chunks.
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="text"
)

# Chains
map_reduce_chain = MapReduceDocumentsChain(
    # Summary small chunks
    llm_chain=map_chain,
    # Combine summaries from above and generate final summary
    reduce_documents_chain=combine_documents_chain,
    document_variable_name="text",
    # Check what happened in between
    return_intermediate_steps=True
)

map_result = map_reduce_chain.invoke(pages)

<ipython-input-7-143e3c3c01e8>:28: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain/
  map_reduce_chain = MapReduceDocumentsChain(


In [ ]:
map_result['output_text']

"- The paper examines vulnerabilities in large language models (LLMs) related to their safety alignment, particularly focusing on ASCII art.\n- It introduces ArtPrompt, a novel jailbreak attack that uses ASCII art to bypass safety measures and provoke harmful outputs from LLMs.\n- The Vision-in-Text Challenge (VITC) is established to evaluate LLMs' ability to recognize non-semantic prompts, revealing significant limitations in five state-of-the-art models.\n- ArtPrompt effectively masks sensitive words in prompts, allowing attackers to evade detection and induce unsafe behaviors.\n- The study highlights that individual characters often lack meaning, and understanding relies more on their arrangement.\n- Various defense mechanisms against jailbreak attacks are discussed, including detection and mitigation strategies.\n- The effectiveness of ArtPrompt is demonstrated through extensive experiments, showing superior performance compared to existing methods.\n- The paper emphasizes the need

- The paper examines vulnerabilities in large language models (LLMs) related to their safety alignment, particularly focusing on ASCII art.
- It introduces ArtPrompt, a novel jailbreak attack that uses ASCII art to bypass safety measures and provoke harmful outputs from LLMs.
- The Vision-in-Text Challenge (VITC) is established to evaluate LLMs' ability to recognize non-semantic prompts, revealing significant limitations in five state-of-the-art models.
- ArtPrompt effectively masks sensitive words in prompts, allowing attackers to evade detection and induce unsafe behaviors.
- The study highlights that individual characters often lack meaning, and understanding relies more on their arrangement.
- Various defense mechanisms against jailbreak attacks are discussed, including detection and mitigation strategies.
- The effectiveness of ArtPrompt is demonstrated through extensive experiments, showing superior performance compared to existing methods.
- The paper emphasizes the need for improved safety measures in LLMs to prevent misuse and harmful outputs.
- Ethical considerations are raised regarding the potential for LLMs to generate instructions for illegal activities, underscoring the importance of robust defenses.

## Intermediate Steps for Map Reducing

In [ ]:
# Show first intermediate couple steps
for i, content in enumerate(map_result['intermediate_steps'][0:2]):
    print(f"Chunk # {i+1}:")
    print(content)
    print("-"*50)

Chunk # 1:
The paper "ArtPrompt: ASCII Art-based Jailbreak Attacks against Aligned LLMs" explores vulnerabilities in large language models (LLMs) related to their safety alignment. It highlights that existing safety measures primarily focus on semantic interpretation, neglecting alternative representations like ASCII art. The authors introduce a novel attack method, ArtPrompt, which exploits this oversight to bypass safety protocols and induce harmful outputs from LLMs.

**Main Points:**
- Safety is crucial for LLMs, with current techniques relying on semantic interpretation.
- Real-world applications often use ASCII art, which poses a challenge for LLMs.
- The paper presents the Vision-in-Text Challenge (VITC) to assess LLMs' ability to recognize non-semantic prompts.
- Five state-of-the-art LLMs (GPT-3.5, GPT-4, Gemini, Claude, Llama2) struggle with ASCII art prompts.
- ArtPrompt is developed as a jailbreak attack that effectively elicits undesired behaviors from LLMs.
- The attack r

# Extrac Credit

In [ ]:
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.2 MB/s eta 0:0

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
# Load the PDF
loader_1 = PyPDFLoader("2402.11753.pdf")
pages_1 = loader.load()

# Split into smaller chunks for better retrieval, this provides better split then load_and_split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=500)
documents = text_splitter.split_documents(pages_1) # gives about 17 chunks

In [ ]:
# Generate embeddings
embeddings = OpenAIEmbeddings(openai_api_key=open_ai_key)

# Store in ChromaDB
vector_db = Chroma.from_documents(documents, embeddings)

In [ ]:
vector_db.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Retrieve relevant document chunks
retrieved_docs = vector_db.similarity_search("Summary this paper in 50 words?", k=2)

# Extract content from retrieved docs
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# Define LLM and prompt
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_key=open_ai_key)
prompt_template = """
Write a concise summary of the paper below, avoid using technical words, list the main points of the paper in bullet points, within 200 words.
  ```{context}```
"""
prompt = PromptTemplate.from_template(prompt_template)

# Run LLM, providing query and retrieved docs in prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)
summary = llm_chain.run({"context": context})

print(summary)


**Summary of the Paper:**

The paper discusses vulnerabilities in large language models (LLMs) when interpreting training data solely based on semantics. It introduces a benchmark called the Vision-in-Text Challenge (VITC) to assess LLMs' ability to recognize prompts that should not be interpreted literally. The authors demonstrate that existing LLMs struggle with this task, leading to potential security risks. They also present a novel attack method, ArtPrompt, which exploits these vulnerabilities to provoke unsafe behaviors in LLMs.

**Main Points:**

- LLMs are vulnerable to attacks when trained only on semantic interpretations.
- The Vision-in-Text Challenge (VITC) benchmark evaluates LLMs' recognition capabilities.
- Five state-of-the-art LLMs were tested, revealing poor performance in recognizing specific prompts.
- ArtPrompt is a new method designed to exploit these vulnerabilities effectively.
- The study highlights the need for improved safety measures in LLM training and alig

**Summary of the Paper:**

The paper discusses vulnerabilities in large language models (LLMs) when interpreting training data solely based on semantics. It introduces a benchmark called the Vision-in-Text Challenge (VITC) to assess LLMs' ability to recognize prompts that should not be interpreted literally. The authors demonstrate that existing LLMs struggle with this task, leading to potential security risks. They also present a novel attack method, ArtPrompt, which exploits these vulnerabilities to provoke unsafe behaviors in LLMs.

**Main Points:**

- LLMs are vulnerable to attacks when trained only on semantic interpretations.
- The Vision-in-Text Challenge (VITC) benchmark evaluates LLMs' recognition capabilities.
- Five state-of-the-art LLMs were tested, revealing poor performance in recognizing specific prompts.
- ArtPrompt is a new method designed to exploit these vulnerabilities effectively.
- The study highlights the need for improved safety measures in LLM training and alignment.
- Future work is suggested to explore the effectiveness of ArtPrompt on multimodal models.
- The paper aims to enhance LLM safety and acknowledges the potential misuse of its findings.